In [ ]:
import pandas as pd
import numpy as np

In [ ]:

train = pd.read_csv('../input/iba-ml1-mid-project/train.csv')
test = pd.read_csv('../input/iba-ml1-mid-project/test.csv')

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
train.shape

In [ ]:
train.dtypes

In [ ]:
train['credit_line_utilization']=train['credit_line_utilization'].apply(lambda x: float(str(x).replace(",",".")))
test['credit_line_utilization']=test['credit_line_utilization'].apply(lambda x: float(str(x).replace(",",".")))

In [ ]:
test.columns

In [ ]:
train.columns

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from itertools import combinations
import warnings
warnings.filterwarnings(action='ignore')
outlier = pd.DataFrame()

perc  = [.05,.25,.75,.995]
for i in perc:
    outlier['PCTL' + '_' + str(i)] = train.quantile(i)
 
    
outlier['upper_cap_condition'] = np.where((outlier['PCTL_0.75']-outlier['PCTL_0.25'])>0,\
                 (outlier['PCTL_0.25']+2.5*(outlier['PCTL_0.75']-outlier['PCTL_0.25'])),outlier['PCTL_0.995'])
outlier['upper_cap']=outlier[['upper_cap_condition','PCTL_0.995']].max(axis=1)
outlier['lower_cap_condition'] = np.where((outlier['PCTL_0.75']-outlier['PCTL_0.25'])>0,\
                 (outlier['PCTL_0.25']-2.5*(outlier['PCTL_0.75']-outlier['PCTL_0.25'])),outlier['PCTL_0.05'])
outlier['lower_cap']=outlier[['lower_cap_condition','PCTL_0.05']].max(axis=1)


dummy_list =[np.nan]

columns =['age', 'number_dependent_family_members', 'monthly_income',
       'number_of_credit_lines', 'real_estate_loans',
       'ratio_debt_payment_to_income', 'credit_line_utilization',
       'number_of_previous_late_payments_up_to_59_days',
       'number_of_previous_late_payments_up_to_89_days',
       'number_of_previous_late_payments_90_days_or_more']

for col in columns:
    train[col] = np.where(~train[col].isin(dummy_list)&(train[col]<outlier.loc[col,'lower_cap']),outlier.loc[col,'lower_cap'],train[col])
    train[col] = np.where(~train[col].isin(dummy_list)&(train[col]>outlier.loc[col,'upper_cap']),outlier.loc[col,'upper_cap'],train[col])

    test[col] = np.where(~test[col].isin(dummy_list)&(test[col]<outlier.loc[col,'lower_cap']),outlier.loc[col,'lower_cap'],test[col])
    test[col] = np.where(~test[col].isin(dummy_list)&(test[col]>outlier.loc[col,'upper_cap']),outlier.loc[col,'upper_cap'],test[col])


In [ ]:
train.isnull().sum()

In [ ]:
columns =['age', 'number_dependent_family_members', 'monthly_income',
       'number_of_credit_lines', 'real_estate_loans',
       'ratio_debt_payment_to_income', 'credit_line_utilization',
       'number_of_previous_late_payments_up_to_59_days',
       'number_of_previous_late_payments_up_to_89_days',
       'number_of_previous_late_payments_90_days_or_more']

In [ ]:
def impute_missing():
    numeric_var = columns
    
    for var in numeric_var:    
        x = train['{}'.format(var)].copy()
        x.dropna(inplace=True)
        x = np.array(x)
        avg = x.mean()
        
        zz = {np.nan:avg}  
        
        train.replace({'{}'.format(var):zz},inplace=True)
        test.replace({'{}'.format(var):zz},inplace=True)


impute_missing()

In [ ]:
train.head()

In [ ]:
train.groupby(by=['number_of_previous_late_payments_up_to_59_days']).agg({"Id":"count","defaulted_on_loan":["sum","mean"]})

In [ ]:
train.number_of_previous_late_payments_up_to_59_days.value_counts()

In [ ]:
train['util_bin']= pd.cut(train.credit_line_utilization,30) 
train['bin_90']= pd.cut(train.number_of_previous_late_payments_90_days_or_more,4)
train['bin_59']= pd.cut(train.number_of_previous_late_payments_up_to_59_days,4) 
train['bin_89']= pd.cut(train.number_of_previous_late_payments_up_to_89_days,4) 

In [ ]:
def woe_ordered_continuous(name, df, discrete_variabe_name, good_bad_variable_df):
        df = pd.concat([df[discrete_variabe_name], good_bad_variable_df], axis = 1)
        df = pd.concat([df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].count(),
                        df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].mean()], axis = 1)
        df = df.iloc[:, [0, 1, 3]]
        df.columns = [df.columns.values[0], 'n_obs', 'prop_good']
        df['prop_n_obs'] = df['n_obs'] / df['n_obs'].sum()
        df['n_good'] = df['prop_good'] * df['n_obs']
        df['n_bad'] = (1 - df['prop_good']) * df['n_obs']
        df['prop_n_good'] = df['n_good'] / df['n_good'].sum()
        df['prop_n_bad'] = df['n_bad'] / df['n_bad'].sum()
        df[name] = np.log(df['prop_n_bad'] / df['prop_n_good'])
        return df

In [ ]:
df1= woe_ordered_continuous('util_bin_1', train, 'util_bin', train.defaulted_on_loan)

In [ ]:
a1= list(df1.util_bin)
a2= list(df1.util_bin_1)

def find_bin(x):
  for i in range(len(a1)):
    if x in a1[i]:
      return a2[i]
  return x

In [ ]:
train['util_woe']= train['credit_line_utilization'].apply(find_bin)
test['util_woe']= test['credit_line_utilization'].apply(find_bin)

In [ ]:
df2= woe_ordered_continuous('bin_90_1', train, 'bin_90', train.defaulted_on_loan)

In [ ]:
a1= list(df2.bin_90)
a2= list(df2.bin_90_1)

def find_bin2(x):
  for i in range(len(a1)):
    if x in a1[i]:
      return a2[i]
  return x

In [ ]:
train['bin_90_woe']= train['number_of_previous_late_payments_90_days_or_more'].apply(find_bin2)
test['bin_90_woe']= test['number_of_previous_late_payments_90_days_or_more'].apply(find_bin2)

In [ ]:
df3= woe_ordered_continuous('bin_59_1', train, 'bin_59', train.defaulted_on_loan)

In [ ]:
a1= list(df3.bin_59)
a2= list(df3.bin_59_1)

def find_bin3(x):
  for i in range(len(a1)):
    if x in a1[i]:
      return a2[i]
  return x

In [ ]:
train['bin_59_woe']= train['number_of_previous_late_payments_up_to_59_days'].apply(find_bin3)
test['bin_59_woe']= test['number_of_previous_late_payments_up_to_59_days'].apply(find_bin3)

In [ ]:
df4= woe_ordered_continuous('bin_89_1', train, 'bin_89', train.defaulted_on_loan)

In [ ]:
a1= list(df4.bin_89)
a2= list(df4.bin_89_1)

def find_bin4(x):
  for i in range(len(a1)):
    if x in a1[i]:
      return a2[i]
  return x

In [ ]:
train['bin_89_woe']= train['number_of_previous_late_payments_up_to_89_days'].apply(find_bin4)
test['bin_89_woe']= test['number_of_previous_late_payments_up_to_89_days'].apply(find_bin4)

In [ ]:
train.columns

In [ ]:
train.drop(columns=['util_bin', 'bin_90', 'bin_59', 'bin_89'], inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    train, train.defaulted_on_loan, test_size=0.2, random_state=42)

In [ ]:
columns =[ 'age', 'number_dependent_family_members', 'monthly_income',
       'number_of_credit_lines', 'real_estate_loans',
       'ratio_debt_payment_to_income', 'credit_line_utilization',
       'number_of_previous_late_payments_up_to_59_days',
       'number_of_previous_late_payments_up_to_89_days',
       'number_of_previous_late_payments_90_days_or_more',
       'util_woe', 'bin_90_woe', 'bin_59_woe', 'bin_89_woe']
# linear regression feature importance
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot
# define dataset
X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, random_state=1)
# define the model
model = LinearRegression()
# fit the model
model.fit(X_train[columns], X_train.defaulted_on_loan)
# get importance
importance = model.coef_
# summarize feature importance
for i,v, in enumerate(importance):
	print('Feature: %0d, Score: %.5f %s' % (i,v, columns[i]))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
cols =['age',
 'number_dependent_family_members',
 'monthly_income',
 'number_of_credit_lines',
 'real_estate_loans',
 'ratio_debt_payment_to_income',
 'credit_line_utilization',
 'number_of_previous_late_payments_up_to_59_days',
 'number_of_previous_late_payments_up_to_89_days',
 'number_of_previous_late_payments_90_days_or_more']

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from itertools import combinations
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
dd= ['real_estate_loans','credit_line_utilization','number_of_previous_late_payments_up_to_59_days',
       'number_of_previous_late_payments_up_to_89_days',
       'number_of_previous_late_payments_90_days_or_more']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=4, random_state=0,min_samples_split=4, min_samples_leaf=1)
clf.fit(X_train[columns], X_train.defaulted_on_loan)
predict = clf.predict_proba(X_test[columns])
predict2 = clf.predict(X_test[columns])
arc = roc_auc_score(X_test.defaulted_on_loan, predict[:,1])
arc

In [ ]:
clf1=LogisticRegression().fit(X_train[dd], X_train.defaulted_on_loan)
predict = clf1.predict_proba(X_test[dd])
predict2 = clf1.predict(X_test[dd])
arc = roc_auc_score(X_test.defaulted_on_loan, predict[:,1])
arc

In [ ]:
clf1=LogisticRegression().fit(X_train[columns[-6:-1]], X_train.defaulted_on_loan)
predict = clf1.predict_proba(X_test[columns[-6:-1]])
predict2 = clf1.predict(X_test[columns[-6:-1]])
arc = roc_auc_score(X_test.defaulted_on_loan, predict[:,1])
arc

In [ ]:
train.columns

In [ ]:
dd=['number_of_previous_late_payments_up_to_59_days', 'real_estate_loans',
 'monthly_income', 'number_of_previous_late_payments_90_days_or_more',
 'number_of_credit_lines', 'number_of_previous_late_payments_up_to_89_days',
 'credit_line_utilization', 'age']

In [ ]:
X_train.replace([np.inf, -np.inf],value=0, inplace=True)
X_test.replace([np.inf, -np.inf],value=0, inplace=True)

In [ ]:
clf1=LogisticRegression().fit(X_train[dd], X_train.defaulted_on_loan)
predict = clf1.predict_proba(X_test[dd])
predict2 = clf1.predict(X_test[dd])

In [ ]:
len(columns)

In [ ]:
train.columns

In [ ]:
columns=['age', 'number_dependent_family_members', 'monthly_income',
       'number_of_credit_lines', 'real_estate_loans',
       'ratio_debt_payment_to_income', 'credit_line_utilization',
       'number_of_previous_late_payments_up_to_59_days',
       'number_of_previous_late_payments_up_to_89_days',
       'number_of_previous_late_payments_90_days_or_more',
       'util_woe', 'bin_90_woe', 'bin_59_woe', 'bin_89_woe']

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for i in columns:
  scaler.fit(X_train[[i]])
  X_train[i]=scaler.transform(X_train[[i]])
  X_test[i]=scaler.transform(X_test[[i]])
  test[i]= scaler.transform(test[[i]])

In [ ]:
max=0
comb=[]
for i in range(500):
  cols= np.random.choice(columns, 7)
  clf1=LogisticRegression().fit(X_train[cols], X_train.defaulted_on_loan)
  predict = clf1.predict_proba(X_test[cols])
  predict2 = clf1.predict(X_test[cols])
  arc = roc_auc_score(X_test.defaulted_on_loan, predict[:,1])
  #dev_gini= (arc*2)-1
  if max<arc:
    max= arc
    comb= cols
    print(i, max)
  #print(cols,  arc)
  
print(comb, max)

In [ ]:
clf1=LogisticRegression().fit(X_train[dd], X_train.defaulted_on_loan)
predict = clf1.predict_proba(X_test[dd])
predict2 = clf1.predict(X_test[dd])

In [ ]:
arc = roc_auc_score(X_test.defaulted_on_loan, predict[:,1])
dev_gini= (arc*2)-1
print(arc)
print(dev_gini)

In [ ]:
columns

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier    
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error,classification_report
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import plot_confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
def p(a):
    print(a)


In [ ]:
model1 = Pipeline(steps=[
    ('impute', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('clf', DecisionTreeClassifier()),   
])
#
param_space = {
    'impute__strategy': ['mean', 'median','most_frequent'],
    'clf__max_depth': range(1, 20, 1)
}
#
gridsearch1 = GridSearchCV(model1, param_space, cv=5)

#
gridsearch1.fit(X_train[columns], X_train.defaulted_on_loan)
#
print(gridsearch1.best_params_)
#
print('Grids Best Score with TreeClf: ' , gridsearch1.best_score_)

In [ ]:
predict = gridsearch1.predict_proba(X_test[columns])

In [ ]:
arc = roc_auc_score(X_test.defaulted_on_loan, predict[:,1])
dev_gini= (arc*2)-1
print(arc)
print(dev_gini)

In [ ]:
from sklearn.metrics import f1_score
f1_score(X_test.defaulted_on_loan, predict2, average='micro')

In [ ]:
predict3= clf.predict_proba(test[columns])
test['Predicted']=predict3[:,1]

In [ ]:
predict3

In [ ]:
test.head()

In [ ]:
submission = test[['Id','Predicted']]
submission.to_csv('submission_v9.csv', index=False)

In [ ]:
submission.head()

In [ ]:
cols =['age',
 'number_dependent_family_members',
 'monthly_income',
 'number_of_credit_lines',
 'real_estate_loans',
 'ratio_debt_payment_to_income',
 'credit_line_utilization',
 'number_of_previous_late_payments_up_to_59_days',
 'number_of_previous_late_payments_up_to_89_days',
 'number_of_previous_late_payments_90_days_or_more']

In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
#from catboost import CatBoostClassifier

In [ ]:
!pip install pycaret

In [ ]:
from pycaret.classification import *

In [ ]:
aa=list(X_train.columns)[1:]

In [ ]:
aa

In [ ]:
train.columns

In [ ]:
bb=['age', 'number_dependent_family_members', 'monthly_income',
       'number_of_credit_lines', 'real_estate_loans',
       'ratio_debt_payment_to_income', 'credit_line_utilization',
       'number_of_previous_late_payments_up_to_59_days',
       'number_of_previous_late_payments_up_to_89_days',
       'number_of_previous_late_payments_90_days_or_more', 'defaulted_on_loan',
       'util_woe', 'bin_90_woe', 'bin_59_woe', 'bin_89_woe']

In [ ]:
bb=['ratio_debt_payment_to_income','monthly_income','age','credit_line_utilization','defaulted_on_loan']

In [ ]:
clf = setup(data = X_train[bb], target = 'defaulted_on_loan', fix_imbalance=True)

In [ ]:
cat = create_model('lightgbm')

In [ ]:
plot_model(cat, plot='feature')

In [ ]:

test_pred = predict_model(cat, data=X_test[bb])
#subm = pd.DataFrame({'ID': train['ID'],'true': train['Top-up Month'],'Top-up Month': test_pred['Label']})

In [ ]:
test_pred

In [ ]:
test_pred
arc = roc_auc_score(X_test.defaulted_on_loan, test_pred['Label'])
dev_gini= (arc*2)-1
print(arc)
print(dev_gini)